In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter
from torchsummary import summary 
torch.__version__

'1.8.1+cpu'

In [6]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader的功能就是把数据加载进来，设置成想要的格式然后按batchsize 分割好

In [7]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),#把灰度映射到0到1
                           transforms.Normalize((0.1307,), (0.3081,))#对图像数据进行标准化 后面两个参数分别是各个通道的均值和标准差
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

每个data里有512张图片，整个train_loader里一共有60000张图片 分成118块

In [8]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [11]:
writer= SummaryWriter()

数据集可视化

In [12]:
data_batch,label_batch=next(iter(train_loader))
img_grid=torchvision.utils.make_grid(data_batch,nrow=32,normalize=True,scale_each=True)
writer.add_image('input_image_2',img_grid,0)

dataset 里有固定的测试集合

构建网络

In [2]:
#structure 1
class ConvNet1(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 50) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(50, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [119]:
#structure 2
class ConvNet2(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 1, 3) # 输入通道数1，输出通道数10，核的大小5
        self.conv3 = nn.Conv2d(1, 10, 3)
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 50) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(50, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out=self.conv3(out)
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [130]:
# structure 3
class ConvNet3(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 50) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(50, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = torch.sigmoid(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = torch.sigmoid(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = torch.sigmoid(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

构造训练测试函数

In [31]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    accuracy=0
    train_loss=0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #这行代码的意思是将所有最开始读取数据时的tensor变量copy一份到device所指定的GPU上去，之后的运算都在GPU上进行
        optimizer.zero_grad()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        train_loss+=loss
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: #118组数据每次取30相当于1/4
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss/=len(train_loader.dataset)
    accuracy=correct/len(train_loader.dataset)
    writer.add_scalar('loss/epoch',train_loss.item(), epoch)
    #writer.add_scalar('train/epoch',accuracy, epoch)

In [32]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    accuracy=0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy=100. * correct / len(test_loader.dataset)
    test_loss /= len(test_loader.dataset)#输出的是平均损失
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.add_scalar('test_loss/epoch',test_loss,epoch)
    writer.add_scalar('test/epoch',accuracy, epoch)

In [24]:
# 返回一个epoch内iteration的图像
def train_withinepoch(model, device, train_loader, optimizer, epoch):
    model.train()
    
    accuracy=0
    for batch_idx, (data, target) in enumerate(train_loader):
        correct = 0
        data, target = data.to(device), target.to(device)
        #这行代码的意思是将所有最开始读取数据时的tensor变量copy一份到device所指定的GPU上去，之后的运算都在GPU上进行
        optimizer.zero_grad()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        accuracy=correct/len(data)   
        writer.add_scalar('act_train_loss/iteration',loss.item(), batch_idx)
        writer.add_scalar('act_train_accu/iteration',accuracy, batch_idx)
        if(batch_idx+1)%30 == 0: #118组数据每次取30相当于1/4
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))   

In [13]:
model1 = ConvNet1().to(DEVICE)
optimizer1 = optim.Adam(model1.parameters())

In [122]:
model2 = ConvNet2().to(DEVICE)
optimizer2 = optim.Adam(model2.parameters())

In [131]:
model3 = ConvNet3().to(DEVICE)
optimizer3 = optim.Adam(model3.parameters())

In [134]:
print(summary(model1,(1,28,28),device='cpu'))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2           [-1, 20, 10, 10]           1,820
            Linear-3                   [-1, 50]         100,050
            Linear-4                   [-1, 10]             510
Total params: 102,640
Trainable params: 102,640
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.39
Estimated Total Size (MB): 0.45
----------------------------------------------------------------
None


In [135]:
print(summary(model2,(1,28,28),device='cpu'))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 26, 26]              10
            Conv2d-2           [-1, 10, 24, 24]             100
            Conv2d-3           [-1, 20, 10, 10]           1,820
            Linear-4                   [-1, 50]         100,050
            Linear-5                   [-1, 10]             510
Total params: 102,490
Trainable params: 102,490
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.39
Estimated Total Size (MB): 0.46
----------------------------------------------------------------
None


In [136]:
print(summary(model3,(1,28,28),device='cpu'))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2           [-1, 20, 10, 10]           1,820
            Linear-3                   [-1, 50]         100,050
            Linear-4                   [-1, 10]             510
Total params: 102,640
Trainable params: 102,640
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.39
Estimated Total Size (MB): 0.45
----------------------------------------------------------------
None


structure 1 的表现

In [160]:
with SummaryWriter('D:\\Document\\python_text\\runs'+'\\Linear') as writer:
    for epoch in range(1, EPOCHS + 1):#可以看出一个epoch就是数据集上的所有样本都跑完一遍
        train(model1, DEVICE, train_loader, optimizer1, epoch)
        
        test(model1, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.430253
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.315362
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.277270

Test set: Average loss: 0.1690, Accuracy: 9518/10000 (95%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.102387
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.089983
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.107243

Test set: Average loss: 0.0882, Accuracy: 9747/10000 (97%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.099168
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.090412
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.055350

Test set: Average loss: 0.0604, Accuracy: 9793/10000 (98%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.068430
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.067210
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.065809

Test set: Average loss: 0.0579, Accuracy: 9808/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.040630
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.039512
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.079267

Test 

In [192]:
for epoch in range(1, EPOCHS + 1):#可以看出一个epoch就是数据集上的所有样本都跑完一遍
        train(model3, DEVICE, train_loader, optimizer3, epoch)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 2.289826
Train Epoch: 1 [30208/60000 (50%)]	Loss: 2.077784
Train Epoch: 1 [45568/60000 (75%)]	Loss: 1.407144
Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.762222
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.602912
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.539983
Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.458538
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.388022
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.401562


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12845056 bytes. Buy new RAM!

比较relu函数和sigmoid函数的差别

In [13]:
with SummaryWriter(conmment='activation_diff') as writer:
        train_withinepoch(model1, DEVICE, train_loader, optimizer1, 1)
        train_withinepoch(model3, DEVICE, train_loader, optimizer, 1)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.464572
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.296220
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.237985


跑完之后需要把数据删掉

比较structure 1和structure 2之间的区别

In [81]:
model1 = ConvNet1().to(DEVICE)
optimizer1 = optim.Adam(model1.parameters())

在第一个epoch内随着iteration的表现

In [69]:
with SummaryWriter(comment='struc_diff') as writer:
        train_withinepoch(model1, DEVICE, train_loader, optimizer1, 1)
        train_withinepoch(model2, DEVICE, train_loader, optimizer9, 1)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.380029
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.293711
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.187705


In [ ]:
model2 = ConvNet2().to(DEVICE)
optimizer2 = optim.Adam(model2.parameters())
model1 = ConvNet1().to(DEVICE)
optimizer1 = optim.Adam(model1.parameters())

在20个epoch内的表现

In [166]:
with SummaryWriter(comment='struc_diff') as writer:
    for epoch in range(1, EPOCHS + 1):#可以看出一个epoch就是数据集上的所有样本都跑完一遍
        train(model4, DEVICE, train_loader, optimizer4, epoch)        
        test(model4, DEVICE, test_loader)
    for epoch in range(1, EPOCHS + 1):
        train(model2, DEVICE, train_loader, optimizer2, epoch)        
        test(model2, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.004774
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.002575
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.008070

Test set: Average loss: 0.0350, Accuracy: 9896/10000 (99%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.004603
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.009482
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.008202

Test set: Average loss: 0.0353, Accuracy: 9894/10000 (99%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.006560
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.001629
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.001703

Test set: Average loss: 0.0310, Accuracy: 9903/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.003929
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.010865
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.001647

Test set: Average loss: 0.0346, Accuracy: 9901/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.006202
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.002303
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.001865

Test 

In [17]:
#writer=SummaryWriter('D:\\Document\\python_text\\runs'+'demo1')

用hook函数提取特征图

In [15]:
data_batch,label_batch=next(iter(train_loader))

In [16]:
model1(data_batch).size()

torch.Size([512, 10])

In [17]:
label_batch

tensor([3, 5, 0, 3, 8, 2, 9, 4, 9, 8, 3, 4, 8, 3, 3, 1, 6, 7, 5, 4, 1, 4, 2, 4,
        4, 4, 4, 3, 3, 2, 4, 7, 0, 8, 9, 1, 7, 8, 0, 2, 0, 2, 1, 7, 3, 6, 1, 8,
        4, 0, 3, 6, 6, 7, 6, 3, 4, 2, 7, 4, 7, 1, 5, 6, 8, 0, 3, 5, 6, 1, 7, 4,
        2, 1, 4, 2, 5, 1, 5, 9, 4, 1, 8, 4, 4, 7, 1, 8, 6, 4, 1, 4, 5, 1, 8, 1,
        9, 1, 8, 3, 9, 3, 1, 7, 6, 1, 9, 0, 1, 8, 8, 7, 2, 7, 0, 6, 7, 6, 3, 5,
        3, 6, 0, 1, 3, 6, 4, 4, 7, 8, 3, 7, 6, 8, 3, 6, 4, 0, 9, 3, 8, 9, 0, 6,
        8, 4, 9, 2, 1, 1, 5, 1, 6, 5, 4, 1, 1, 7, 5, 3, 5, 1, 1, 5, 4, 7, 4, 9,
        4, 6, 2, 0, 8, 6, 2, 5, 5, 2, 3, 5, 7, 5, 1, 9, 7, 3, 3, 6, 7, 3, 9, 6,
        6, 7, 0, 4, 6, 2, 0, 6, 6, 1, 2, 6, 0, 8, 3, 3, 6, 5, 4, 1, 2, 6, 3, 3,
        8, 6, 6, 2, 8, 8, 9, 5, 9, 3, 9, 4, 5, 5, 8, 9, 1, 5, 6, 1, 8, 6, 2, 7,
        8, 5, 8, 2, 9, 7, 2, 2, 8, 3, 8, 8, 5, 3, 0, 8, 7, 7, 5, 3, 5, 8, 0, 5,
        1, 8, 4, 2, 5, 9, 4, 9, 4, 1, 3, 7, 4, 4, 2, 6, 5, 5, 1, 0, 2, 4, 1, 9,
        2, 4, 6, 4, 6, 7, 6, 1, 9, 4, 7,

In [18]:
data1=data_batch[1].unsqueeze(0)

提取三个structure的特征图

In [19]:
#提取model1的特征图
def forward_hook(module, data_input, data_output):
        fmap_block.append(data_output)
        input_block.append(data_input)

In [20]:
fmap_block = list()
input_block = list()
model1.conv2.register_forward_hook(forward_hook)
output = model1(data1)
a=fmap_block[0].transpose(0,1)
with SummaryWriter(comment='struc_diff') as writer:   
    fmap_grid = torchvision.utils.make_grid(a, normalize=True, scale_each=True, nrow=10)
    writer.add_image('feature_map1 ', fmap_grid, global_step=321)

In [202]:
fmap_block = list()
input_block = list()
model2.conv3.register_forward_hook(forward_hook)
output = model2(data1)
a=fmap_block[0].transpose(0,1)
with SummaryWriter(comment='struc_diff') as writer:   
    fmap_grid = torchvision.utils.make_grid(a, normalize=True, scale_each=True, nrow=10)
    writer.add_image('feature_map2 ', fmap_grid, global_step=321)

In [203]:
fmap_block = list()
input_block = list()
model3.conv2.register_forward_hook(forward_hook)
output = model3(data1)
a=fmap_block[0].transpose(0,1)
with SummaryWriter(comment='struc_diff') as writer:   
    fmap_grid = torchvision.utils.make_grid(a, normalize=True, scale_each=True, nrow=10)
    writer.add_image('feature_map3 ', fmap_grid, global_step=321)

In [199]:
writer.close()